In [1]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. **Pre-training σε μεγάλα unlabelled datasets**:

  Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. **Fine-tuning σε μικρότερα labelled datasets**:
  
     Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του **text-classification pipeline** μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης.

Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες: **[entailment/neutral/contradiction]**.

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι **γραμματικά ορθό (acceptable) ή όχι (unacceptable)**:

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια.
Το  Yelp κατασκευάστηκε θεωρώντας τα αστέρια 1 και 2 αρνητικά και τα 3 και 4 θετικά.  Η αρνητική πολικότητα ανήκει στην κατηγορία 1 και η θετική στην κατηγορία 2. Τα reviews αυτά χωρίζονται σε αυτές τις κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.



In [2]:
from datasets import load_dataset

# insert your code here
dataset = load_dataset("yelp_polarity")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.93k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [12]:
# insert your code here
train_labels = set(dataset['train']['label'])
print("Categories in train set:", train_labels)

test_labels = set(dataset['test']['label'])
print("Categories in test set:", test_labels)

train_dataset = dataset["train"].shuffle(seed=42).select(range(300))
test_dataset = dataset["test"].shuffle(seed=42).select(range(300))

Categories in train set: {0, 1}
Categories in test set: {0, 1}


In [13]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

In [14]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα.

Η διαδικασία αυτή επιτελείται μέσω των **Tokenizers**, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, **αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer**. Επιλέγοντας τη μέθοδο **from_pretrained** λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [15]:
# insert your code here
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [16]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# insert your code here
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [17]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})

In [18]:
test_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [19]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε;  Σχολιάστε και αναλύστε.

In [20]:
from transformers import TrainingArguments, Trainer, get_scheduler
from torch.optim import AdamW
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model.device)

all_evaluations = []
# Hyperparameters
num_epochs = [3, 4, 10]
learning_rates = [2e-5, 3e-5, 5e-5]
batch_sizes = [4, 16, 32]

for epoch in num_epochs:
    for lr in learning_rates:
        for bs in batch_sizes:
            try:
                print(f"\nRun: epochs={epoch}, lr={lr}, bs={bs}")

                arguments = TrainingArguments(
                    output_dir = "test_trainer",
                    num_train_epochs = epoch,
                    learning_rate = lr,
                    per_device_train_batch_size = bs,
                    eval_strategy = "epoch",
                    save_strategy = "epoch",
                    load_best_model_at_end = True,
                    metric_for_best_model = "accuracy",
                    logging_dir = "./logs",
                    report_to = "none",
                    dataloader_num_workers = 2,
                    fp16 = True,
                    )

                trainer = Trainer(
                    model = model,
                    args = arguments,
                    train_dataset = train_dataset,
                    eval_dataset = test_dataset,
                    compute_metrics = compute_metrics,
                )

                optimizer = AdamW(model.parameters(), lr = lr)

                num_training_steps = len(train_dataset) // bs * epoch

                scheduler = get_scheduler(
                    "linear",
                    optimizer = optimizer,
                    num_warmup_steps = 0,
                    num_training_steps = num_training_steps,
                )

                trainer.optimizer = optimizer
                trainer.lr_scheduler = scheduler

                print(f"Training with {epoch} epochs...")
                trainer.train()
                print("Evaluating...")
                evaluations = trainer.evaluate()
                results = {
                    "Learning Rate": lr,
                    "Batch Size": bs,
                    "Epochs": epoch,
                    "Accuracy": evaluations["eval_accuracy"]
                }
                all_evaluations.append(results)

            except Exception as e:
              print(f"Failed with ep={epoch}, lr={lr}, bs={bs}: {str(e)}")
              continue

cuda:0

Run: epochs=3, lr=2e-05, bs=4
Training with 3 epochs...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.286752,0.896667
2,No log,0.372531,0.890000
3,No log,0.382636,0.896667


Evaluating...



Run: epochs=3, lr=2e-05, bs=16
Training with 3 epochs...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.341198,0.863333
2,No log,0.286870,0.910000
3,No log,0.303627,0.910000


...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.137033,0.893333
2,No log,2.143439,0.893333
3,No log,2.148596,0.893333
4,No log,2.099574,0.890000
5,No log,2.090706,0.886667
6,No log,2.114049,0.890000
7,No log,2.120079,0.893333
8,No log,2.122139,0.893333
9,No log,2.122756,0.893333
10,No log,2.123848,0.893333


Evaluating...



Run: epochs=10, lr=5e-05, bs=32
Training with 10 epochs...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.140523,0.893333
2,No log,2.143908,0.893333
3,No log,2.146825,0.893333
4,No log,3.151239,0.836667
5,No log,2.353090,0.873333
6,No log,2.382498,0.860000
7,No log,2.594704,0.853333
8,No log,2.339504,0.866667
9,No log,2.308995,0.870000
10,No log,2.308443,0.870000


Evaluating...


In [23]:
all_evaluations

[{'Learning Rate': 2e-05,
  'Batch Size': 4,
  'Epochs': 3,
  'Accuracy': 0.8966666666666666},
 {'Learning Rate': 2e-05, 'Batch Size': 16, 'Epochs': 3, 'Accuracy': 0.91},
 {'Learning Rate': 2e-05,
  'Batch Size': 32,
  'Epochs': 3,
  'Accuracy': 0.9133333333333333},
 {'Learning Rate': 3e-05,
  'Batch Size': 4,
  'Epochs': 3,
  'Accuracy': 0.9033333333333333},
 {'Learning Rate': 3e-05, 'Batch Size': 16, 'Epochs': 3, 'Accuracy': 0.89},
 {'Learning Rate': 3e-05, 'Batch Size': 32, 'Epochs': 3, 'Accuracy': 0.89},
 {'Learning Rate': 5e-05,
  'Batch Size': 4,
  'Epochs': 3,
  'Accuracy': 0.8966666666666666},
 {'Learning Rate': 5e-05,
  'Batch Size': 16,
  'Epochs': 3,
  'Accuracy': 0.8933333333333333},
 {'Learning Rate': 5e-05, 'Batch Size': 32, 'Epochs': 3, 'Accuracy': 0.89},
 {'Learning Rate': 2e-05,
  'Batch Size': 4,
  'Epochs': 4,
  'Accuracy': 0.8933333333333333},
 {'Learning Rate': 2e-05,
  'Batch Size': 16,
  'Epochs': 4,
  'Accuracy': 0.8933333333333333},
 {'Learning Rate': 2e-05,
  

---
Τα αποτελέσματα του fine-tuning μας επιτρέπουν να εντοπίσουμε ποιοι συνδυασμοί υπερπαραμέτρων οδηγούν στη βέλτιστη απόδοση. Τα αποτελέσματα αυτά είναι:

*   Learning Rate: 2e-05
*   Batch Size: 4
*   Epochs: 3
*   **Accurancy**: **0.8967**
---
*   Learning Rate: 2e-05
*   Batch Size: 16
*   Epochs: 3
*   **Accurancy**: **0.9100**
---
*   Learning Rate: 2e-05
*   Batch Size: 32
*   Epochs: 3
*   **Accurancy**: **0.9133**
---
*   Learning Rate: 3e-05
*   Batch Size: 4
*   Epochs: 3
*   **Accurancy**: **0.9033**
---
*   Learning Rate: 3e-05
*   Batch Size: 16
*   Epochs: 3
*   **Accurancy**: **0.8900**
---
*   Learning Rate: 3e-05
*   Batch Size: 32
*   Epochs: 3
*   **Accurancy**: **0.8900**
---
*   Learning Rate: 5e-05
*   Batch Size: 4
*   Epochs: 3
*   **Accurancy**: **0.8967**
---
*   Learning Rate: 5e-05
*   Batch Size: 16
*   Epochs: 3
*   **Accurancy**: **0.8933**
---
*   Learning Rate: 5e-05
*   Batch Size: 32
*   Epochs: 3
*   **Accurancy**: **0.8900**

**Σημείωση**: Όλοι οι συνδυασμοί με Epochs = 4 και Epochs = 10 οδήγησαν στο ίδιο **Accuracy** = **0.8933**.


---



---
### Learning Rate
* Το μικρότερο learning rate (2e-05) πέτυχε τα καλύτερα αποτελέσματα, ειδικά με batch sizes 16 και 32.

* Το μεγαλύτερο learning rate (5e-05) είχε γενικά χειρότερες επιδόσεις, δείχνοντας πιθανή αστάθεια κατά την εκπαίδευση.

* Το 3e-05 έδωσε μέτρια αποτελέσματα και σταθερότητα, αλλά όχι το μέγιστο accuracy.

### Batch Size
* Οι μεγαλύτερες τιμές batch size (16 και 32) συνοδεύτηκαν από υψηλότερα accuracy, ιδίως σε συνδυασμό με learning rate 2e-05.

* Το μικρό batch size (4) δεν ήταν αποτελεσματικό σε γενικές γραμμές.

### Epochs
* Όλοι οι συνδυασμοί με 4 ή 10 epochs καταλήγουν σε πανομοιότυπο accuracy (0.8933), πράγμα που δείχνει ότι το μοντέλο είτε έχει φτάσει σε κορεσμό πολύ νωρίς (π.χ. στις 3 εποχές), είτε δεν μαθαίνει περαιτέρω λόγω χαμηλού learning rate ή άλλων περιορισμών.

---



# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή λογική (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [24]:
# # insert your code here (load dataset)
piqa_dataset = load_dataset("piqa")

README.md:   0%|          | 0.00/8.41k [00:00<?, ?B/s]

piqa.py:   0%|          | 0.00/5.36k [00:00<?, ?B/s]

The repository for piqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/piqa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [86]:
# insert your code here (models)
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer, get_scheduler
import torch

train_dataset = piqa_dataset["train"].shuffle(seed=42).select(range(100))
test_dataset = piqa_dataset["test"].shuffle(seed=42).select(range(100))

model_names = [
    "bert-base-uncased",
    "roberta-large",
    "albert-base-v2",
    "xlnet-base-cased",
    "distilbert-base-uncased"
]

tokenizers = [AutoTokenizer.from_pretrained(model_name) for model_name in model_names]
models = [AutoModelForMultipleChoice.from_pretrained(model_name) for model_name in model_names]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLNetForMultipleChoice were not initialized from the model checkpoint a

In [87]:
# insert code
from evaluate import load

class Transformer:
    def __init__(self, model_name, dataset, num_samples = 100):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForMultipleChoice.from_pretrained(model_name)
        self.dataset = dataset['train'].select(range(num_samples))
        self.metric = load("accuracy")

# insert code
    def preprocess(self, examples):
        first_sentences = [[context] * 2 for context in examples["goal"]]
        questions = examples["sol1"], examples["sol2"]
        second_sentences = [
            [header for header in question_header] for question_header in zip(*questions)
        ]

        first_sentences = sum(first_sentences, [])
        second_sentences = sum(second_sentences, [])

        tokenized_examples = self.tokenizer(first_sentences, second_sentences, truncation = True, padding = True)

        return {
            k: [v[i:i + 2] for i in range(0, len(v), 2)]
            for k, v in tokenized_examples.items()
        }

    def compute_metrics(self, p):
        preds = p.predictions.argmax(-1)
        return self.metric.compute(predictions = preds, references = p.label_ids)

    def train(self):
        tokenized_datasets = self.dataset.map(self.preprocess, batched = True)

        arguments = TrainingArguments(
            eval_strategy = "epoch",
            learning_rate = 2e-5,
            num_train_epochs = 3,
            output_dir = "./results",
            per_device_eval_batch_size = 16,
            per_device_train_batch_size = 16,
            report_to = "none",
        )

        trainer = Trainer(
            model =  self.model,
            args = arguments,
            train_dataset = tokenized_datasets,
            eval_dataset = tokenized_datasets,
            tokenizer = self.tokenizer,
            compute_metrics = self.compute_metrics,
        )

        trainer.train()
        return trainer.evaluate()

In [85]:
for model_name in model_names:
    mc_model = Transformer(model_name, piqa_dataset)
    results = mc_model.train()
    print(f"Model: {model_name} - Results: {results}")

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

<ipython-input-84-909dab3ccd46>:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.685156,0.700000
2,No log,0.679160,0.780000
3,No log,0.676052,0.800000


Model: bert-base-uncased - Results: {'eval_loss': 0.6760522723197937, 'eval_accuracy': 0.8, 'eval_runtime': 0.3387, 'eval_samples_per_second': 295.239, 'eval_steps_per_second': 20.667, 'epoch': 3.0}


...

Some weights of DistilBertForMultipleChoice were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

<ipython-input-84-909dab3ccd46>:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.691792,0.600000
2,No log,0.691055,0.660000
3,No log,0.690654,0.670000


Model: distilbert-base-uncased - Results: {'eval_loss': 0.6906542778015137, 'eval_accuracy': 0.67, 'eval_runtime': 0.1876, 'eval_samples_per_second': 532.963, 'eval_steps_per_second': 37.307, 'epoch': 3.0}


---
Τα αποτελέσματα είναι:

### bert-base-uncased
*   Eval Loss: 0.6761
*   Eval Accuracy: 0.80

### roberta-large
*   Eval Loss: 0.6912
*   Eval Accuracy: 0.73

### albert-base-v2
*   Eval Loss: 0.6656
*   Eval Accuracy: 0.78

### xlnet-base-cased
*   Eval Loss: 0.6892
*   Eval Accuracy: 0.49

### distilbert-base-uncased
*   Eval Loss: 0.6907
*   Eval Accuracy: 0.67

---



## B2. Truthful QA

### Sentence Transformers

Οι **sentence transformers** χρησιμοποιούνται για να δημιουργήσουν **embeddings προτάσεων**, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [88]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [89]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

np.float32(0.4048847)

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [91]:
# insert your code here (load dataset)
from datasets import load_dataset

truthful_qa_dataset = load_dataset("truthful_qa", "generation")
truthful_qa_dataset_val = truthful_qa_dataset['validation']

In [94]:
from sentence_transformers import SentenceTransformer
import numpy as np

# insert your code here (load models for semantic similarity and QA)
filtered_truthful_qa_dataset_val = [item for item in truthful_qa_dataset_val if len(item['correct_answers']) >= 2][:100]

model_names = [
    'sentence-transformers/all-mpnet-base-v2',
    'sentence-transformers/all-distilroberta-v1',
    'sentence-transformers/all-MiniLM-L6-v2',
    'sentence-transformers/paraphrase-MiniLM-L12-v2',
    'sentence-transformers/quora-distilbert-multilingual',
    'sentence-transformers/stsb-roberta-base-v2'
]

# insert your code here (function for optimal correct answers & semantic similarity)
def calculate_accuracy(model_name, data):
    model = SentenceTransformer(model_name)
    correct_count = 0 # Counter for correctly matched answers

    for item in data:
        question = item['question']
        best_answer = item['best_answer']
        correct_answers = item['correct_answers'][:2]   # take 2 correct answers
        incorrect_answer = item['incorrect_answers'][0] # take 1 incorrect answer

        # list of answers to compare: best_answer, two correct, and one incorrect
        candidates = [best_answer] + correct_answers + [incorrect_answer]
        embeddings = model.encode(candidates)

        best_embedding = embeddings[0]
        correct_embeddings = embeddings[1:3]

        # cosine similarity between best_answer and each of the correct answers
        predicted_index = np.argmax([cosine_similarity([best_embedding], [emb])[0][0] for emb in correct_embeddings])

        if cosine_similarity([best_embedding], [correct_embeddings[predicted_index]])[0][0] > 0.95:
            correct_count += 1

    return correct_count / len(data)


accuracies = {}
for model_name in model_names:
    accuracy = calculate_accuracy(model_name, filtered_truthful_qa_dataset_val)
    accuracies[model_name] = accuracy

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [95]:
accuracies

{'sentence-transformers/all-mpnet-base-v2': 0.83,
 'sentence-transformers/all-distilroberta-v1': 0.81,
 'sentence-transformers/all-MiniLM-L6-v2': 0.81,
 'sentence-transformers/paraphrase-MiniLM-L12-v2': 0.81,
 'sentence-transformers/quora-distilbert-multilingual': 0.91,
 'sentence-transformers/stsb-roberta-base-v2': 0.78}

---
Τα αποτελέσματα είναι:

### Accurancy
*   all-mpnet-base-v2: 0.83
*   all-distilroberta-v1: 0.81
*   all-MiniLM-L6-v2: 0.81
*   paraphrase-MiniLM-L12-v2: 0.81
*   quora-distilbert-multilingual: 0.81
*   stsb-roberta-base-v2: 0.78

---

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [5]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [34]:
# insert your code here (load dataset)
from datasets import load_dataset

dataset = load_dataset("winogrande", "winogrande_xl")
sub_dataset = dataset["train"].shuffle(seed=42).select(range(100))

Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [35]:
# insert your code here (load models)
# insert your code here (create pipelines)
from transformers import pipeline

model_names = [
    "roberta-large",
    "bert-large-uncased-whole-word-masking",
    "distilbert-base-uncased"
]

pipelines = {model_name: pipeline("fill-mask", model=model_name) for model_name in model_names}

Device set to use cpu
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Device set to use cpu


In [37]:
from transformers import pipeline
from datasets import load_dataset

# insert your code here (function for predicting best fill)
def predict_best_fill(model_pipeline, question, choices, model_name):

    if "roberta" in model_name.lower():
        mask_token = "<mask>"
    else:
        mask_token = "[MASK]"

    question_with_mask = question.replace("_", mask_token)
    predictions = [model_pipeline(question_with_mask.replace(mask_token, f"{mask_token} {choice}"))[0] for choice in choices]

    best_choice = max(predictions, key=lambda x: x['score'])
    return best_choice['token_str'], best_choice['score']

def calculate_accuracy(model_pipeline, data, model_name):
    correct_count = 0

    for item in data:
        sentence = item['sentence']
        option1 = item['option1']
        option2 = item['option2']

        best_fill, _ = predict_best_fill(model_pipeline, sentence, [option1, option2], model_name)
        correct_answer_index = 0 if item['answer'] == '1' else 1

        if best_fill == [option1, option2][correct_answer_index]:
            correct_count += 1

    return correct_count / len(data)

In [39]:
accuracies = {}
for model_name in model_names:
    model_pipeline = pipelines[model_name]  # Use the pre-loaded pipeline
    accuracy = calculate_accuracy(model_pipeline, sub_dataset, model_name)
    accuracies[model_name] = accuracy

# Display the accuracies for each model
print(accuracies)

{'roberta-large': 0.0, 'bert-large-uncased-whole-word-masking': 0.03, 'distilbert-base-uncased': 0.01}
